In [ ]:
"""
Analysis of Hyperdirect Pathway Connectivity and Beta Suppression
in STN-DBS Treatment Response

Required packages:
pip install pandas numpy scipy statsmodels scikit-learn matplotlib seaborn
pip install pingouin>=0.5.3  # For mediation analysis with BCa bootstrap
pip install statsmodels>=0.14.0  # For mixed models and FDR correction
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# ================== PART 1: MEDIATION ANALYSIS FUNCTIONS ==================

def run_mediation_analysis_complete(data, predictor, mediator, outcome, covariate, n_bootstrap=5000):
    print(f"\n{'='*70}")
    print(f"MEDIATION ANALYSIS - Baron and Kenny Framework with BCa Bootstrap")
    print(f"Predictor: {predictor} → Mediator: {mediator} → Outcome: {outcome}")
    print(f"Covariate: {covariate}")
    print(f"{'='*70}\n")
    
    # Use pingouin's mediation_analysis which implements Baron & Kenny with BCa bootstrap
    # This matches our methodology exactly
    try:
        # FIXED: Set return_dist=False to avoid tuple unpacking issues
        stats_df = pg.mediation_analysis(
            data=data,
            x=predictor,         # Independent variable (HDP connectivity)
            m=mediator,          # Mediator (Beta suppression)
            y=outcome,           # Dependent variable (UPDRS change)
            covar=covariate,     # Covariate (Dosage)
            n_boot=n_bootstrap,  # 5000 bootstrap samples as specified
            seed=42,             # For reproducibility
            return_dist=False
        )
    except Exception as e:
        print(f"Error in mediation analysis: {e}")
        return None
    
    # Extract key statistics from the stats DataFrame
    # Path a: predictor -> mediator (controlling for covariate)
    # Row label is 'M ~ X' where M is mediator and X is predictor
    path_a_row = f'{mediator} ~ {predictor}'
    if path_a_row in stats_df.index:
        path_a_coeff = stats_df.loc[path_a_row, 'coef']
        path_a_se = stats_df.loc[path_a_row, 'se']
        path_a_pval = stats_df.loc[path_a_row, 'pval']
        path_a_ci = [stats_df.loc[path_a_row, 'CI[2.5%]'], stats_df.loc[path_a_row, 'CI[97.5%]']]
    else:
        # Fallback to first row which should be path a
        path_a_coeff = stats_df.iloc[0]['coef']
        path_a_se = stats_df.iloc[0]['se']
        path_a_pval = stats_df.iloc[0]['pval']
        path_a_ci = [stats_df.iloc[0]['CI[2.5%]'], stats_df.iloc[0]['CI[97.5%]']]
    
    print(f"Path a ({predictor} → {mediator}, {covariate}):")
    print(f"  Coefficient: {path_a_coeff:.4f} (SE: {path_a_se:.4f})")
    print(f"  95% CI: [{path_a_ci[0]:.4f}, {path_a_ci[1]:.4f}]")
    print(f"  p-value: {path_a_pval:.4f}")
    
    # Path b: mediator -> outcome (controlling for predictor and covariate)
    # Row label is 'Y ~ M' where Y is outcome and M is mediator
    path_b_row = f'{outcome} ~ {mediator}'
    if path_b_row in stats_df.index:
        path_b_coeff = stats_df.loc[path_b_row, 'coef']
        path_b_se = stats_df.loc[path_b_row, 'se']
        path_b_pval = stats_df.loc[path_b_row, 'pval']
        path_b_ci = [stats_df.loc[path_b_row, 'CI[2.5%]'], stats_df.loc[path_b_row, 'CI[97.5%]']]
    else:
        # Fallback to second row which should be path b
        path_b_coeff = stats_df.iloc[1]['coef']
        path_b_se = stats_df.iloc[1]['se']
        path_b_pval = stats_df.iloc[1]['pval']
        path_b_ci = [stats_df.iloc[1]['CI[2.5%]'], stats_df.iloc[1]['CI[97.5%]']]
    
    print(f"\nPath b ({mediator} → {outcome}, {predictor}, {covariate}):")
    print(f"  Coefficient: {path_b_coeff:.4f} (SE: {path_b_se:.4f})")
    print(f"  95% CI: [{path_b_ci[0]:.4f}, {path_b_ci[1]:.4f}]")
    print(f"  p-value: {path_b_pval:.4f}")
    
    # Path c': Direct effect (predictor -> outcome controlling for mediator and covariate)
    # This is the ADE (Average Direct Effect)
    # Row label is 'Direct'
    if 'Direct' in stats_df.index:
        path_c_prime_coeff = stats_df.loc['Direct', 'coef']
        path_c_prime_se = stats_df.loc['Direct', 'se']
        path_c_prime_pval = stats_df.loc['Direct', 'pval']
        path_c_prime_ci = [stats_df.loc['Direct', 'CI[2.5%]'], stats_df.loc['Direct', 'CI[97.5%]']]
    else:
        # Fallback
        path_c_prime_coeff = stats_df.iloc[3]['coef']
        path_c_prime_se = stats_df.iloc[3]['se']
        path_c_prime_pval = stats_df.iloc[3]['pval']
        path_c_prime_ci = [stats_df.iloc[3]['CI[2.5%]'], stats_df.iloc[3]['CI[97.5%]']]
    
    print(f"\nPath c' - ADE (Average Direct Effect):")
    print(f"  {predictor} → {outcome}, {mediator}, {covariate}")
    print(f"  β = {path_c_prime_coeff:.4f} (SE: {path_c_prime_se:.4f})")
    print(f"  95% CI: [{path_c_prime_ci[0]:.4f}, {path_c_prime_ci[1]:.4f}]")
    print(f"  p-value (raw): {path_c_prime_pval:.4f}")
    
    # ACME (Average Causal Mediation Effect) - this is the indirect effect (a × b)
    # Row label is 'Indirect'
    if 'Indirect' in stats_df.index:
        acme_coeff = stats_df.loc['Indirect', 'coef']
        acme_se = stats_df.loc['Indirect', 'se']
        acme_pval = stats_df.loc['Indirect', 'pval']
        acme_ci = [stats_df.loc['Indirect', 'CI[2.5%]'], stats_df.loc['Indirect', 'CI[97.5%]']]
    else:
        # Fallback
        acme_coeff = stats_df.iloc[4]['coef']
        acme_se = stats_df.iloc[4]['se']
        acme_pval = stats_df.iloc[4]['pval']
        acme_ci = [stats_df.iloc[4]['CI[2.5%]'], stats_df.iloc[4]['CI[97.5%]']]
    
    print(f"\nACME (Average Causal Mediation Effect) - Indirect Effect (a × b):")
    print(f"  β = {acme_coeff:.4f} (SE: {acme_se:.4f})")
    print(f"  95% BCa CI: [{acme_ci[0]:.4f}, {acme_ci[1]:.4f}]")
    print(f"  p-value (raw): {acme_pval:.4f}")
    
    # Total Effect
    # Row label is 'Total'
    if 'Total' in stats_df.index:
        total_effect_coeff = stats_df.loc['Total', 'coef']
        total_effect_se = stats_df.loc['Total', 'se']
        total_effect_pval = stats_df.loc['Total', 'pval']
        total_effect_ci = [stats_df.loc['Total', 'CI[2.5%]'], stats_df.loc['Total', 'CI[97.5%]']]
    else:
        # Fallback
        total_effect_coeff = stats_df.iloc[2]['coef']
        total_effect_se = stats_df.iloc[2]['se']
        total_effect_pval = stats_df.iloc[2]['pval']
        total_effect_ci = [stats_df.iloc[2]['CI[2.5%]'], stats_df.iloc[2]['CI[97.5%]']]
    
    print(f"\nTotal Effect (c = c' + ab):")
    print(f"  β = {total_effect_coeff:.4f} (SE: {total_effect_se:.4f})")
    print(f"  95% CI: [{total_effect_ci[0]:.4f}, {total_effect_ci[1]:.4f}]")
    print(f"  p-value (raw): {total_effect_pval:.4f}")
    
    # Proportion Mediated
    # Note: pingouin calculates this as indirect/total when both have same sign
    if abs(total_effect_coeff) > 0.001:
        prop_mediated = acme_coeff / total_effect_coeff
        print(f"\nProportion Mediated:")
        print(f"  Value: {prop_mediated:.3f} ({prop_mediated*100:.1f}%)")
    else:
        prop_mediated = np.nan
        print(f"\nProportion Mediated: Cannot calculate (total effect ≈ 0)")
    
    # Test significance based on CI
    if acme_ci[0] > 0 or acme_ci[1] < 0:
        print("\nConclusion: SIGNIFICANT mediation (BCa CI excludes zero)")
    else:
        print("\nConclusion: No significant mediation (BCa CI includes zero)")
    
    # Store all p-values for FDR correction later
    p_values = {
        'path_a': path_a_pval,
        'path_b': path_b_pval,
        'acme': acme_pval,
        'ade': path_c_prime_pval,
        'total': total_effect_pval
    }
    
    # Return comprehensive results
    return {
        'path_a': {'coeff': path_a_coeff, 'se': path_a_se, 'pval': path_a_pval, 'ci': path_a_ci},
        'path_b': {'coeff': path_b_coeff, 'se': path_b_se, 'pval': path_b_pval, 'ci': path_b_ci},
        'path_c_prime': {'coeff': path_c_prime_coeff, 'se': path_c_prime_se, 'pval': path_c_prime_pval, 'ci': path_c_prime_ci},
        'acme': {'coeff': acme_coeff, 'se': acme_se, 'pval': acme_pval, 'ci': acme_ci},
        'ade': {'coeff': path_c_prime_coeff, 'se': path_c_prime_se, 'pval': path_c_prime_pval, 'ci': path_c_prime_ci},
        'total_effect': {'coeff': total_effect_coeff, 'se': total_effect_se, 'pval': total_effect_pval, 'ci': total_effect_ci},
        'prop_mediated': prop_mediated,
        'p_values': p_values,
        'pingouin_results': stats_df
    }

# ================== PART 2: TEMPORAL COMPARISON FUNCTION ==================

def compare_mediation_effects_paired_bootstrap(df, predictor, mediator_3m, mediator_6m, 
                                             outcome_3m, outcome_6m, covariate_3m, covariate_6m, 
                                             n_bootstrap=5000):

    print(f"\n{'='*70}")
    print(f"TEMPORAL COMPARISON - Paired Bootstrap (5000 resamples)")
    print(f"Predictor: {predictor}")
    print(f"Mediators: {mediator_3m} (3m), {mediator_6m} (6m)")
    print(f"METHODOLOGY: Testing if indirect effects differ between 3 and 6 months")
    print(f"{'='*70}\n")
    
    # Prepare paired data
    required_cols_3m = [predictor, mediator_3m, outcome_3m, covariate_3m, 'Patient_ID']
    required_cols_6m = [predictor, mediator_6m, outcome_6m, covariate_6m, 'Patient_ID']
    
    # Get complete cases for each timepoint
    data_3m = df[required_cols_3m].dropna()
    data_6m = df[required_cols_6m].dropna()
    
    # Find patients with data at both timepoints
    patients_3m = set(data_3m['Patient_ID'].unique())
    patients_6m = set(data_6m['Patient_ID'].unique())
    common_patients = list(patients_3m.intersection(patients_6m))
    
    print(f"Patients with 3-month data: {len(patients_3m)}")
    print(f"Patients with 6-month data: {len(patients_6m)}")
    print(f"Patients with paired data: {len(common_patients)}")
    
    if len(common_patients) < 5:
        print("ERROR: Insufficient paired data for temporal comparison")
        return None
    
    # Filter to common patients only
    data_3m_paired = data_3m[data_3m['Patient_ID'].isin(common_patients)].copy()
    data_6m_paired = data_6m[data_6m['Patient_ID'].isin(common_patients)].copy()
    
    # Calculate observed indirect effects
    print("\nCalculating observed indirect effects...")
    
    # 3-month mediation
    try:
        stats_3m = pg.mediation_analysis(
            data=data_3m_paired,
            x=predictor,
            m=mediator_3m,
            y=outcome_3m,
            covar=covariate_3m,
            n_boot=5000,
            seed=42,
            return_dist=False
        )
        # Extract indirect effect
        if 'Indirect' in stats_3m.index:
            obs_ie_3m = stats_3m.loc['Indirect', 'coef']
        else:
            obs_ie_3m = stats_3m.iloc[4]['coef']  # Fallback to 5th row
    except Exception as e:
        print(f"Error in 3-month mediation: {e}")
        return None
    
    # 6-month mediation
    try:
        stats_6m = pg.mediation_analysis(
            data=data_6m_paired,
            x=predictor,
            m=mediator_6m,
            y=outcome_6m,
            covar=covariate_6m,
            n_boot=5000,
            seed=42,
            return_dist=False
        )
        # Extract indirect effect
        if 'Indirect' in stats_6m.index:
            obs_ie_6m = stats_6m.loc['Indirect', 'coef']
        else:
            obs_ie_6m = stats_6m.iloc[4]['coef']  # Fallback to 5th row
    except Exception as e:
        print(f"Error in 6-month mediation: {e}")
        return None
    
    # Observed difference
    obs_diff = obs_ie_6m - obs_ie_3m
    
    print(f"\nObserved indirect effects:")
    print(f"  3-month: {obs_ie_3m:.4f}")
    print(f"  6-month: {obs_ie_6m:.4f}")
    print(f"  Difference (6m - 3m): {obs_diff:.4f}")
    
    # Paired bootstrap
    print(f"\nPerforming {n_bootstrap} paired bootstrap iterations...")
    print("(Resampling patients to preserve within-patient correlation)")
    
    boot_differences = []
    successful_boots = 0
    
    for i in range(n_bootstrap):
        # Resample patients with replacement
        bootstrap_patients = np.random.choice(common_patients, 
                                            size=len(common_patients), 
                                            replace=True)
        
        # Create bootstrap datasets by selecting resampled patients
        boot_data_3m = []
        boot_data_6m = []
        
        for patient in bootstrap_patients:
            # Get this patient's data at both timepoints
            patient_data_3m = data_3m_paired[data_3m_paired['Patient_ID'] == patient]
            patient_data_6m = data_6m_paired[data_6m_paired['Patient_ID'] == patient]
            
            boot_data_3m.append(patient_data_3m)
            boot_data_6m.append(patient_data_6m)
        
        boot_data_3m = pd.concat(boot_data_3m, ignore_index=True)
        boot_data_6m = pd.concat(boot_data_6m, ignore_index=True)
        
        try:
            # Run mediation on bootstrap samples
            boot_stats_3m = pg.mediation_analysis(
                data=boot_data_3m,
                x=predictor,
                m=mediator_3m,
                y=outcome_3m,
                covar=covariate_3m,
                n_boot=5000,
                seed=42+i,
                return_dist=False
            )
            
            boot_stats_6m = pg.mediation_analysis(
                data=boot_data_6m,
                x=predictor,
                m=mediator_6m,
                y=outcome_6m,
                covar=covariate_6m,
                n_boot=5000,
                seed=42+i,
                return_dist=False
            )
            
            # Extract indirect effects
            if 'Indirect' in boot_stats_3m.index:
                boot_ie_3m = boot_stats_3m.loc['Indirect', 'coef']
            else:
                boot_ie_3m = boot_stats_3m.iloc[4]['coef']
                
            if 'Indirect' in boot_stats_6m.index:
                boot_ie_6m = boot_stats_6m.loc['Indirect', 'coef']
            else:
                boot_ie_6m = boot_stats_6m.iloc[4]['coef']
            
            # Calculate difference
            boot_diff = boot_ie_6m - boot_ie_3m
            boot_differences.append(boot_diff)
            successful_boots += 1
            
        except Exception as e:
            # Handle any convergence issues
            continue
        
        # Progress indicator
        if (i + 1) % 100 == 0:
            print(f"  Completed {i + 1}/{n_bootstrap} bootstrap iterations...")
    
    boot_differences = np.array(boot_differences)
    
    print(f"\nSuccessful bootstrap iterations: {successful_boots}/{n_bootstrap}")
    
    if len(boot_differences) < 100:
        print("WARNING: Many bootstrap iterations failed. Results may be unstable.")
    
    # Calculate statistics from bootstrap distribution
    # Two-tailed p-value
    p_value = np.mean(np.abs(boot_differences) >= np.abs(obs_diff))
    
    # 95% CI from bootstrap percentiles
    ci_boot = np.percentile(boot_differences, [2.5, 97.5])
    
    # Bootstrap SE
    se_boot = np.std(boot_differences)
    
    # Display results
    print(f"\n{'='*60}")
    print("PAIRED BOOTSTRAP RESULTS")
    print(f"{'='*60}")
    print(f"Difference (6m - 3m): {obs_diff:.4f}")
    print(f"Bootstrap SE: {se_boot:.4f}")
    print(f"95% Bootstrap CI: [{ci_boot[0]:.4f}, {ci_boot[1]:.4f}]")
    print(f"Two-tailed p-value: {p_value:.4f}")
    
    # Interpret results
    if ci_boot[0] > 0 or ci_boot[1] < 0:
        print("\nConclusion: Indirect effects DIFFER significantly between timepoints")
        print("(95% CI excludes zero)")
    else:
        print("\nConclusion: No significant difference in indirect effects")
        print("(95% CI includes zero)")
    
    return {
        'difference': obs_diff,
        'se': se_boot,
        'ci': ci_boot,
        'p_value': p_value,
        'bootstrap_differences': boot_differences,
        'n_patients': len(common_patients),
        'n_successful_boots': successful_boots
    }

# ================== PART 3: CLINICAL PREDICTIVE UTILITY WITH MIXED MODELS ==================

def clinical_predictive_utility_mixed(df, hdp_predictor, hdp_type_name):
    """
    Assess incremental predictive value of HDP beyond levodopa response.
    
    METHODOLOGY SECTION: Stage 3 - Clinical Predictive Utility Analysis
    "Model 1: UPDRS_Change = γ₀ + γ₁·Levodopa_z + γ₂·Time + tau_i + ε
     Model 2: UPDRS_Change = γ₀ + γ₁·Levodopa_z + γ₂·HDP_z + γ₃·Time + tau_i + ε"
    
    Uses mixed linear models with random intercepts (tau_i) as specified.
    Implements leave-one-patient-out cross-validation.
    
    **UPDATED**: Now returns individual patient MSE values for paired plotting.
    """
    print(f"\n{'='*80}")
    print(f"CLINICAL PREDICTIVE UTILITY ANALYSIS - {hdp_type_name}")
    print(f"Using predictor: {hdp_predictor}")
    print(f"METHODOLOGY: Mixed models with random intercepts + LOO-CV")
    print(f"{'='*80}\n")
    
    # Data preparation as specified in methodology
    if df[hdp_predictor].isna().all():
        print(f"WARNING: All values for {hdp_predictor} are NaN. Skipping.")
        return None
    
    # Prepare pooled data with Time coded as 0 (3 months) or 1 (6 months)
    data_3m = df[[hdp_predictor, 'Levodopa_Response', 'UPDRS_Change_3m', 'Patient_ID']].dropna().copy()
    data_3m['Time'] = 0  # As specified in methodology
    data_3m.rename(columns={'UPDRS_Change_3m': 'UPDRS_Change'}, inplace=True)
    
    data_6m = df[[hdp_predictor, 'Levodopa_Response', 'UPDRS_Change_6m', 'Patient_ID']].dropna().copy()
    data_6m['Time'] = 1  # As specified in methodology
    data_6m.rename(columns={'UPDRS_Change_6m': 'UPDRS_Change'}, inplace=True)
    
    pooled_data = pd.concat([data_3m, data_6m], ignore_index=True)
    
    print(f"Sample sizes: 3-month n={len(data_3m)}, 6-month n={len(data_6m)}, Pooled n={len(pooled_data)}")
    
    if len(pooled_data) < 10:
        print(f"WARNING: Insufficient data (n={len(pooled_data)}). Skipping.")
        return None
    
    # Standardize predictors (z-score) as specified
    scaler = StandardScaler()
    pooled_data['HDP_z'] = scaler.fit_transform(pooled_data[[hdp_predictor]])
    pooled_data['Levodopa_z'] = scaler.fit_transform(pooled_data[['Levodopa_Response']])
    
    # Save scalers for sensitivity analysis
    hdp_scaler = StandardScaler().fit(pooled_data[[hdp_predictor]])
    levo_scaler = StandardScaler().fit(pooled_data[['Levodopa_Response']])
    
    # Check multicollinearity using variance_inflation_factor
    print(f"\n{'='*60}")
    print("MULTICOLLINEARITY CHECK")
    print(f"{'='*60}")
    
    # Calculate VIF properly using statsmodels
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    
    X_vif = pooled_data[['Levodopa_z', 'HDP_z']].values
    vif_data = pd.DataFrame()
    vif_data["Variable"] = ['Levodopa_z', 'HDP_z']
    vif_data["VIF"] = [variance_inflation_factor(X_vif, i) for i in range(X_vif.shape[1])]
    
    print("\nVariance Inflation Factors:")
    print(vif_data)
    
    if any(vif_data["VIF"] > 5):
        print("\nWARNING: High multicollinearity detected (VIF > 5)")
    
    # ===== MAIN ANALYSIS: POOLED WITH RANDOM EFFECTS =====
    print(f"\n{'='*60}")
    print("MAIN ANALYSIS: POOLED MIXED MODELS")
    print("METHODOLOGY: Testing γ₂ = 0 with Z-test")
    print(f"{'='*60}")
    
    # Model 1: As specified in methodology
    formula1 = 'UPDRS_Change ~ Levodopa_z + Time'
    model1 = smf.mixedlm(formula1, data=pooled_data, groups=pooled_data['Patient_ID']).fit()
    
    print("\nModel 1 (Levodopa + Time + Random Intercepts):")
    print(f"Formula: {formula1}")
    print(f"Log-likelihood: {model1.llf:.2f}")
    print(f"AIC: {model1.aic:.2f}, BIC: {model1.bic:.2f}")
    
    # Model 2: As specified in methodology
    formula2 = 'UPDRS_Change ~ Levodopa_z + HDP_z + Time'
    model2 = smf.mixedlm(formula2, data=pooled_data, groups=pooled_data['Patient_ID']).fit()
    
    print("\nModel 2 (Levodopa + HDP + Time + Random Intercepts):")
    print(f"Formula: {formula2}")
    print(f"Log-likelihood: {model2.llf:.2f}")
    print(f"AIC: {model2.aic:.2f}, BIC: {model2.bic:.2f}")
    
    # Z-test for HDP coefficient as specified
    hdp_coef = model2.params['HDP_z']
    hdp_se = model2.bse['HDP_z']
    z_stat = hdp_coef / hdp_se
    p_value_z = 2 * (1 - stats.norm.cdf(abs(z_stat)))
    
    print(f"\n{'='*60}")
    print("Z-TEST FOR HDP COEFFICIENT (γ₂ = 0)")
    print(f"{'='*60}")
    print(f"HDP coefficient (γ₂): {hdp_coef:.4f}")
    print(f"Standard error: {hdp_se:.4f}")
    print(f"Z-statistic: {z_stat:.3f}")
    print(f"P-value (raw): {p_value_z:.4f}")
    
    # Variance components as specified
    print(f"\n{'='*60}")
    print("VARIANCE COMPONENTS")
    print(f"{'='*60}")
    random_var = model2.cov_re.iloc[0, 0]
    residual_var = model2.scale
    icc = random_var / (random_var + residual_var)
    print(f"Random Effect Variance (τ²): {random_var:.3f}")
    print(f"Residual Variance (σ²): {residual_var:.3f}")
    print(f"Intraclass Correlation Coefficient (ICC): {icc:.3f}")
    
    # ===== LEAVE-ONE-PATIENT-OUT CROSS-VALIDATION =====
    print(f"\n{'='*60}")
    print("LEAVE-ONE-PATIENT-OUT CROSS-VALIDATION")
    print("METHODOLOGY: Using sklearn's LeaveOneOut for proper implementation")
    print(f"{'='*60}")
    
    # Group by patient for LOO
    patients = pooled_data['Patient_ID'].unique()
    
    # Initialize LOO cross-validator
    loo = LeaveOneOut()
    
    # Store predictions and actuals
    cv_results_model1 = []
    cv_results_model2 = []
    
    # Create patient indices for LOO
    patient_indices = []
    for patient in patients:
        patient_indices.append(pooled_data[pooled_data['Patient_ID'] == patient].index.tolist())
    
    # Perform LOO-CV
    for train_patients, test_patient in loo.split(patients):
        # Get train and test indices
        train_idx = []
        for p_idx in train_patients:
            train_idx.extend(patient_indices[p_idx])
        test_idx = patient_indices[test_patient[0]]
        
        # Split data
        train_data = pooled_data.iloc[train_idx].copy()
        test_data = pooled_data.iloc[test_idx].copy()
        
        if len(test_data) == 0 or len(train_data) < 10:
            continue
        
        try:
            # Fit models on training data
            model1_train = smf.mixedlm(formula1, data=train_data, groups=train_data['Patient_ID']).fit()
            model2_train = smf.mixedlm(formula2, data=train_data, groups=train_data['Patient_ID']).fit()
            
            # Predict for test patient (fixed effects only for new patient)
            # Model 1
            test_design1 = np.column_stack([
                np.ones(len(test_data)),
                test_data['Levodopa_z'].values,
                test_data['Time'].values
            ])
            pred1 = np.dot(test_design1, model1_train.fe_params)
            
            # Model 2
            test_design2 = np.column_stack([
                np.ones(len(test_data)),
                test_data['Levodopa_z'].values,
                test_data['HDP_z'].values,
                test_data['Time'].values
            ])
            pred2 = np.dot(test_design2, model2_train.fe_params)
            
            actual = test_data['UPDRS_Change'].values
            
            # Calculate MSE
            mse1 = mean_squared_error(actual, pred1)
            mse2 = mean_squared_error(actual, pred2)
            
            cv_results_model1.append({'patient': patients[test_patient[0]], 'mse': mse1})
            cv_results_model2.append({'patient': patients[test_patient[0]], 'mse': mse2})
            
        except Exception as e:
            print(f"Error in CV for patient {patients[test_patient[0]]}: {e}")
            continue
    
    cv_df_model1 = pd.DataFrame(cv_results_model1)
    cv_df_model2 = pd.DataFrame(cv_results_model2)
    
    # Print individual patient MSE values for plotting
    print("\n--- INDIVIDUAL PATIENT MSE VALUES ---")
    print("Patient ID, Model 1 (Levo), Model 2 (Levo+HDP)")
    print("-" * 45)
    
    # Merge the dataframes to ensure paired values
    if len(cv_df_model1) > 0 and len(cv_df_model2) > 0:
        cv_merged = pd.merge(cv_df_model1, cv_df_model2, on='patient', suffixes=('_model1', '_model2'))
        for _, row in cv_merged.iterrows():
            print(f"{row['patient']:10}, {row['mse_model1']:.6f}, {row['mse_model2']:.6f}")
        
        mse_pooled_model1 = cv_df_model1['mse'].mean()
        mse_pooled_model2 = cv_df_model2['mse'].mean()
        print(f"\nCross-validation results (n={len(cv_results_model1)} patients):")
        print(f"Mean MSE Model 1 (Levodopa only): {mse_pooled_model1:.6f}")
        print(f"Mean MSE Model 2 (Levodopa + HDP): {mse_pooled_model2:.6f}")
        improvement_pct = (mse_pooled_model1 - mse_pooled_model2) / mse_pooled_model1 * 100
        print(f"Improvement: {improvement_pct:.1f}%")
    else:
        print("\nInsufficient data for pooled cross-validation")
        mse_pooled_model1 = np.nan
        mse_pooled_model2 = np.nan
        cv_merged = pd.DataFrame()
    
    # ===== SENSITIVITY ANALYSIS: SEPARATE TIMEPOINTS =====
    print(f"\n{'='*60}")
    print("SENSITIVITY ANALYSIS: SEPARATE TIMEPOINTS (NO RANDOM EFFECTS)")
    print("METHODOLOGY: OLS at each timepoint separately")
    print(f"{'='*60}")
    
    # 3-month analysis
    print("\n--- 3-MONTH ANALYSIS ---")
    mse_3m_model1 = np.nan
    mse_3m_model2 = np.nan
    p_value_3m = np.nan
    
    if len(data_3m) >= 5:
        # Standardize using pooled scalers
        data_3m['HDP_z'] = hdp_scaler.transform(data_3m[[hdp_predictor]])
        data_3m['Levodopa_z'] = levo_scaler.transform(data_3m[['Levodopa_Response']])
        
        # OLS models (no Time variable as we're analyzing separately)
        X1_3m = sm.add_constant(data_3m[['Levodopa_z']])
        model1_3m = sm.OLS(data_3m['UPDRS_Change'], X1_3m).fit()
        
        X2_3m = sm.add_constant(data_3m[['Levodopa_z', 'HDP_z']])
        model2_3m = sm.OLS(data_3m['UPDRS_Change'], X2_3m).fit()
        
        print(f"Model 1 R²: {model1_3m.rsquared:.3f}")
        print(f"Model 2 R²: {model2_3m.rsquared:.3f}")
        
        # Use t-test p-value from OLS (appropriate for small samples)
        if 'HDP_z' in model2_3m.params:
            hdp_coef_3m = model2_3m.params['HDP_z']
            hdp_se_3m = model2_3m.bse['HDP_z']
            t_stat_3m = model2_3m.tvalues['HDP_z']
            p_value_3m = model2_3m.pvalues['HDP_z']  # This is the t-test p-value
            print(f"HDP coefficient: {hdp_coef_3m:.4f}, t={t_stat_3m:.3f}, p={p_value_3m:.4f}")
        
        # LOO-CV for both models
        loo_3m = LeaveOneOut()
        X_3m_model1 = data_3m[['Levodopa_z']].values
        X_3m_model2 = data_3m[['Levodopa_z', 'HDP_z']].values
        y_3m = data_3m['UPDRS_Change'].values
        
        # Model 1 LOO-CV
        predictions_3m_model1 = []
        for train_idx, test_idx in loo_3m.split(X_3m_model1):
            X_train = sm.add_constant(X_3m_model1[train_idx])
            X_test = sm.add_constant(X_3m_model1[test_idx])
            y_train = y_3m[train_idx]
            model_cv = sm.OLS(y_train, X_train).fit()
            pred = model_cv.predict(X_test)
            predictions_3m_model1.extend(pred)
        
        mse_3m_model1 = mean_squared_error(y_3m, predictions_3m_model1)
        
        # Model 2 LOO-CV
        predictions_3m_model2 = []
        for train_idx, test_idx in loo_3m.split(X_3m_model2):
            X_train = sm.add_constant(X_3m_model2[train_idx])
            X_test = sm.add_constant(X_3m_model2[test_idx])
            y_train = y_3m[train_idx]
            model_cv = sm.OLS(y_train, X_train).fit()
            pred = model_cv.predict(X_test)
            predictions_3m_model2.extend(pred)
        
        mse_3m_model2 = mean_squared_error(y_3m, predictions_3m_model2)
        
        print(f"\n3-month LOO-CV MSE:")
        print(f"  Model 1 (Levodopa only): {mse_3m_model1:.6f}")
        print(f"  Model 2 (Levodopa + HDP): {mse_3m_model2:.6f}")
        print(f"  Improvement: {(mse_3m_model1 - mse_3m_model2) / mse_3m_model1 * 100:.1f}%")
    
    # 6-month analysis
    print("\n--- 6-MONTH ANALYSIS ---")
    mse_6m_model1 = np.nan
    mse_6m_model2 = np.nan
    p_value_6m = np.nan
    
    if len(data_6m) >= 5:
        # Similar analysis for 6 months...
        data_6m['HDP_z'] = hdp_scaler.transform(data_6m[[hdp_predictor]])
        data_6m['Levodopa_z'] = levo_scaler.transform(data_6m[['Levodopa_Response']])
        
        X1_6m = sm.add_constant(data_6m[['Levodopa_z']])
        model1_6m = sm.OLS(data_6m['UPDRS_Change'], X1_6m).fit()
        
        X2_6m = sm.add_constant(data_6m[['Levodopa_z', 'HDP_z']])
        model2_6m = sm.OLS(data_6m['UPDRS_Change'], X2_6m).fit()
        
        print(f"Model 1 R²: {model1_6m.rsquared:.3f}")
        print(f"Model 2 R²: {model2_6m.rsquared:.3f}")
        
        # Use t-test p-value from OLS
        if 'HDP_z' in model2_6m.params:
            hdp_coef_6m = model2_6m.params['HDP_z']
            hdp_se_6m = model2_6m.bse['HDP_z']
            t_stat_6m = model2_6m.tvalues['HDP_z']
            p_value_6m = model2_6m.pvalues['HDP_z']  # This is the t-test p-value
            print(f"HDP coefficient: {hdp_coef_6m:.4f}, t={t_stat_6m:.3f}, p={p_value_6m:.4f}")
        
        # LOO-CV for both models
        loo_6m = LeaveOneOut()
        X_6m_model1 = data_6m[['Levodopa_z']].values
        X_6m_model2 = data_6m[['Levodopa_z', 'HDP_z']].values
        y_6m = data_6m['UPDRS_Change'].values
        
        # Model 1 LOO-CV
        predictions_6m_model1 = []
        for train_idx, test_idx in loo_6m.split(X_6m_model1):
            X_train = sm.add_constant(X_6m_model1[train_idx])
            X_test = sm.add_constant(X_6m_model1[test_idx])
            y_train = y_6m[train_idx]
            model_cv = sm.OLS(y_train, X_train).fit()
            pred = model_cv.predict(X_test)
            predictions_6m_model1.extend(pred)
        
        mse_6m_model1 = mean_squared_error(y_6m, predictions_6m_model1)
        
        # Model 2 LOO-CV
        predictions_6m_model2 = []
        for train_idx, test_idx in loo_6m.split(X_6m_model2):
            X_train = sm.add_constant(X_6m_model2[train_idx])
            X_test = sm.add_constant(X_6m_model2[test_idx])
            y_train = y_6m[train_idx]
            model_cv = sm.OLS(y_train, X_train).fit()
            pred = model_cv.predict(X_test)
            predictions_6m_model2.extend(pred)
        
        mse_6m_model2 = mean_squared_error(y_6m, predictions_6m_model2)
        
        print(f"\n6-month LOO-CV MSE:")
        print(f"  Model 1 (Levodopa only): {mse_6m_model1:.6f}")
        print(f"  Model 2 (Levodopa + HDP): {mse_6m_model2:.6f}")
        print(f"  Improvement: {(mse_6m_model1 - mse_6m_model2) / mse_6m_model1 * 100:.1f}%")
    
    # ===== FINAL COMPARISON =====
    print(f"\n{'='*60}")
    print("FINAL MSE COMPARISON")
    print("METHODOLOGY: Determine best approach (pooled vs separate)")
    print(f"{'='*60}")
    print(f"MSE Pooled Model 2 (Mixed Model): {mse_pooled_model2:.6f}")
    print(f"MSE 3-month Model 2 (OLS): {mse_3m_model2:.6f}")
    print(f"MSE 6-month Model 2 (OLS): {mse_6m_model2:.6f}")
    
    # Determine best approach
    mse_values = {'Pooled': mse_pooled_model2, '3-month': mse_3m_model2, '6-month': mse_6m_model2}
    valid_mse = {k: v for k, v in mse_values.items() if not np.isnan(v)}
    
    if valid_mse:
        best_approach = min(valid_mse, key=valid_mse.get)
        print(f"\nBest approach: {best_approach} (lowest MSE)")
    
    # Collect all p-values for this analysis
    p_values_clinical = {
        'mixed_model': p_value_z,
        '3_month_ols': p_value_3m,
        '6_month_ols': p_value_6m
    }
    
    # Include individual patient MSE data in results
    return {
        'hdp_type': hdp_type_name,
        'models': {'model1': model1, 'model2': model2},
        'z_test': {
            'coefficient': hdp_coef,
            'se': hdp_se,
            'z_stat': z_stat,
            'p_value': p_value_z
        },
        'variance_components': {
            'random_variance': random_var,
            'residual_variance': residual_var,
            'icc': icc
        },
        'mse_comparison': {
            'pooled_model1': mse_pooled_model1,
            'pooled_model2': mse_pooled_model2,
            '3_month_model1': mse_3m_model1,
            '3_month_model2': mse_3m_model2,
            '6_month_model1': mse_6m_model1,
            '6_month_model2': mse_6m_model2
        },
        'cv_results': {
            'pooled_model1': cv_df_model1 if len(cv_df_model1) > 0 else None,
            'pooled_model2': cv_df_model2 if len(cv_df_model2) > 0 else None,
            'pooled_paired': cv_merged if len(cv_merged) > 0 else None  # NEW: paired results
        },
        'p_values': p_values_clinical
    }

# ================== FDR CORRECTION FUNCTION ==================

def apply_fdr_correction(all_p_values, alpha=0.05):
    """
    Apply False Discovery Rate correction to all p-values.
    
    METHODOLOGY NOTE: This extends our analysis plan to include FDR
    in addition to Bonferroni correction, providing better balance
    between Type I and Type II error control.
    """
    print(f"\n{'='*80}")
    print("MULTIPLE COMPARISON CORRECTION")
    print(f"{'='*80}\n")
    
    # Flatten all p-values into categories
    mediation_pvals = []
    temporal_pvals = []
    clinical_pvals = []
    
    labels_mediation = []
    labels_temporal = []
    labels_clinical = []
    
    for key, pval_dict in all_p_values['mediation'].items():
        if pval_dict is not None:
            mediation_pvals.extend([
                pval_dict.get('path_a', np.nan),
                pval_dict.get('path_b', np.nan),
                pval_dict.get('acme', np.nan),
                pval_dict.get('ade', np.nan),
                pval_dict.get('total', np.nan)
            ])
            labels_mediation.extend([
                f"{key}_path_a",
                f"{key}_path_b",
                f"{key}_acme",
                f"{key}_ade",
                f"{key}_total"
            ])
    
    for key, pval in all_p_values['temporal'].items():
        if pval is not None:
            temporal_pvals.append(pval)
            labels_temporal.append(key)
    
    for key, pval_dict in all_p_values['clinical'].items():
        if pval_dict is not None:
            clinical_pvals.extend([
                pval_dict.get('mixed_model', np.nan),
                pval_dict.get('3_month_ols', np.nan),
                pval_dict.get('6_month_ols', np.nan)
            ])
            labels_clinical.extend([
                f"{key}_mixed",
                f"{key}_3m",
                f"{key}_6m"
            ])
    
    # Apply FDR correction within each family
    results = {}
    
    # Mediation tests
    if len(mediation_pvals) > 0:
        mediation_pvals_clean = [p for p in mediation_pvals if not np.isnan(p)]
        labels_clean = [l for l, p in zip(labels_mediation, mediation_pvals) if not np.isnan(p)]
        
        if len(mediation_pvals_clean) > 0:
            reject, pvals_fdr, _, _ = multipletests(mediation_pvals_clean, alpha=alpha, method='fdr_bh')
            print("MEDIATION ANALYSIS - FDR Correction (Benjamini-Hochberg):")
            print(f"Number of tests: {len(mediation_pvals_clean)}")
            print(f"FDR threshold: {alpha}")
            results['mediation_fdr'] = dict(zip(labels_clean, pvals_fdr))
            # Show significant results
            sig_count = sum(reject)
            print(f"Significant after FDR: {sig_count}/{len(mediation_pvals_clean)}")
    
    # Temporal comparisons
    if len(temporal_pvals) > 0:
        temporal_pvals_clean = [p for p in temporal_pvals if not np.isnan(p)]
        labels_clean = [l for l, p in zip(labels_temporal, temporal_pvals) if not np.isnan(p)]
        
        if len(temporal_pvals_clean) > 0:
            reject, pvals_fdr, _, _ = multipletests(temporal_pvals_clean, alpha=alpha, method='fdr_bh')
            print("\nTEMPORAL COMPARISONS - FDR Correction:")
            print(f"Number of tests: {len(temporal_pvals_clean)}")
            results['temporal_fdr'] = dict(zip(labels_clean, pvals_fdr))
            print(f"Significant after FDR: {sum(reject)}/{len(temporal_pvals_clean)}")
    
    # Clinical utility
    if len(clinical_pvals) > 0:
        clinical_pvals_clean = [p for p in clinical_pvals if not np.isnan(p)]
        labels_clean = [l for l, p in zip(labels_clinical, clinical_pvals) if not np.isnan(p)]
        
        if len(clinical_pvals_clean) > 0:
            reject, pvals_fdr, _, _ = multipletests(clinical_pvals_clean, alpha=alpha, method='fdr_bh')
            print("\nCLINICAL UTILITY - FDR Correction:")
            print(f"Number of tests: {len(clinical_pvals_clean)}")
            results['clinical_fdr'] = dict(zip(labels_clean, pvals_fdr))
            print(f"Significant after FDR: {sum(reject)}/{len(clinical_pvals_clean)}")
    
    # Also show Bonferroni thresholds as mentioned in methodology
    print("\nBONFERRONI THRESHOLDS (as specified in methodology):")
    print(f"- Mediation: α = 0.05/{len(mediation_pvals_clean) if mediation_pvals_clean else 1} = "
          f"{0.05/len(mediation_pvals_clean) if mediation_pvals_clean else 0.05:.4f}")
    print(f"- Clinical utility: α = 0.05/3 = 0.017")
    
    return results

# ================== VISUALIZATION FUNCTIONS ==================

def create_comprehensive_visualizations(all_results):
    """
    Create comprehensive visualization of all results with both raw and FDR p-values.
    """
    print("\nVisualization creation would include both raw and FDR-corrected p-values...")
    # Implementation omitted for brevity but would follow same structure

# ================== MAIN EXECUTION ==================

def main():
    """
    Execute complete analysis pipeline EXACTLY as specified in methodology.
    Uses established statistical packages throughout.
    """
    print("="*80)
    print("COMPREHENSIVE DBS ANALYSIS")
    print("Using Established Statistical Packages")
    print("="*80)
    print("\nPackages used:")
    print("- pingouin 0.5.3+: Mediation analysis with BCa bootstrap")
    print("- statsmodels 0.14.0+: Mixed models, OLS, VIF, FDR correction")
    print("- scikit-learn 1.0.0+: StandardScaler, LeaveOneOut CV")
    print("- scipy 1.9.0+: Statistical distributions")
    print("- pandas, numpy: Data manipulation")
    print("="*80)
    
    # Load data
    print("\nLoading data...")
    df = pd.read_csv('lfp_dataframe.csv')
    
    # Add Patient_ID if not present
    if 'Patient_ID' not in df.columns:
        df['Patient_ID'] = range(len(df))
    
    # Create synthetic Levodopa_Response if not present
    if 'Levodopa_Response' not in df.columns:
        print("\nWARNING: Creating synthetic Levodopa_Response data")
        np.random.seed(42)
        df['Levodopa_Response'] = 30 + np.random.normal(20, 15, len(df))
        df['Levodopa_Response'] = np.clip(df['Levodopa_Response'], 10, 80)
    
    # Define variables as in methodology
    hdp_types = ['M1', 'SMA', 'PFC']
    hdp_predictors_stage1 = ['HDP_M1_Count', 'HDP_SMA_Count', 'HDP_PFC_Count']
    hdp_predictors_stage3 = ['pre_HDP_M1_Count', 'pre_HDP_SMA_Count', 'pre_HDP_PFC_Count']
    
    beta_mediators_3m = {
        'Total': 'Total_Beta_Suppression_3m',
        'Low': 'Low_Beta_Suppression_3m',
        'High': 'High_Beta_Suppression_3m'
    }
    beta_mediators_6m = {
        'Total': 'Total_Beta_Suppression_6m',
        'Low': 'Low_Beta_Suppression_6m',
        'High': 'High_Beta_Suppression_6m'
    }
    
    # Change Stimulation_Strength columns to Dosage
    df.rename(columns={
        'Stimulation_Strength_3m': 'Dosage_3m',
        'Stimulation_Strength_6m': 'Dosage_6m'
    }, inplace=True)
    
    # Store all results and p-values
    all_results = {
        'mediation_3m': {},
        'mediation_6m': {},
        'temporal_comparisons': {},
        'clinical_utility': {}
    }
    
    all_p_values = {
        'mediation': {},
        'temporal': {},
        'clinical': {}
    }
    
    # ===== STAGE 1 & 2: MEDIATION ANALYSIS =====
    print("\n" + "="*80)
    print("STAGE 1 & 2: MEDIATION ANALYSIS")
    print("METHODOLOGY: Baron & Kenny with BCa bootstrap (5000 resamples)")
    print("="*80)
    
    for hdp_idx, (hdp_predictor, hdp_type) in enumerate(zip(hdp_predictors_stage1, hdp_types)):
        print(f"\n\n{'='*80}")
        print(f"ANALYZING HDP TYPE: {hdp_type}")
        print(f"Using predictor: {hdp_predictor}")
        print(f"{'='*80}")
        
        if hdp_predictor not in df.columns:
            print(f"WARNING: {hdp_predictor} not found in data. Skipping.")
            continue
        
        if df[hdp_predictor].isna().all():
            print(f"WARNING: All values for {hdp_predictor} are NaN. Skipping.")
            continue
        
        # Analyze each beta band
        for beta_type, mediator_3m in beta_mediators_3m.items():
            mediator_6m = beta_mediators_6m[beta_type]
            
            print(f"\n{'='*70}")
            print(f"BETA BAND: {beta_type}")
            print(f"{'='*70}")
            
            # 3-month analysis
            print(f"\n### 3-MONTH ANALYSIS - {hdp_type} → {beta_type} Beta ###")
            required_cols_3m = [hdp_predictor, mediator_3m, 'UPDRS_Change_3m', 'Dosage_3m']
            
            if all(col in df.columns for col in required_cols_3m):
                data_3m = df[required_cols_3m].dropna()
                
                if len(data_3m) >= 5:
                    results_3m = run_mediation_analysis_complete(
                        data_3m,
                        predictor=hdp_predictor,
                        mediator=mediator_3m,
                        outcome='UPDRS_Change_3m',
                        covariate='Dosage_3m'
                    )
                    
                    if results_3m is not None:
                        key = f'{hdp_predictor}_{beta_type}_Beta'
                        all_results['mediation_3m'][key] = results_3m
                        all_p_values['mediation'][f"{key}_3m"] = results_3m['p_values']
                else:
                    print(f"Insufficient data for 3-month analysis (n={len(data_3m)})")
            else:
                print(f"Missing columns for 3-month analysis")
            
            # 6-month analysis
            print(f"\n### 6-MONTH ANALYSIS - {hdp_type} → {beta_type} Beta ###")
            required_cols_6m = [hdp_predictor, mediator_6m, 'UPDRS_Change_6m', 'Dosage_6m']
            
            if all(col in df.columns for col in required_cols_6m):
                data_6m = df[required_cols_6m].dropna()
                
                if len(data_6m) >= 5:
                    results_6m = run_mediation_analysis_complete(
                        data_6m,
                        predictor=hdp_predictor,
                        mediator=mediator_6m,
                        outcome='UPDRS_Change_6m',
                        covariate='Dosage_6m'
                    )
                    
                    if results_6m is not None:
                        key = f'{hdp_predictor}_{beta_type}_Beta'
                        all_results['mediation_6m'][key] = results_6m
                        all_p_values['mediation'][f"{key}_6m"] = results_6m['p_values']
                else:
                    print(f"Insufficient data for 6-month analysis (n={len(data_6m)})")
            else:
                print(f"Missing columns for 6-month analysis")
            
            # Temporal comparison
            key = f'{hdp_predictor}_{beta_type}_Beta'
            if key in all_results['mediation_3m'] and key in all_results['mediation_6m']:
                print(f"\n### TEMPORAL COMPARISON - {hdp_type} → {beta_type} Beta ###")
                
                temporal_results = compare_mediation_effects_paired_bootstrap(
                    df,
                    predictor=hdp_predictor,
                    mediator_3m=mediator_3m,
                    mediator_6m=mediator_6m,
                    outcome_3m='UPDRS_Change_3m',
                    outcome_6m='UPDRS_Change_6m',
                    covariate_3m='Dosage_3m',
                    covariate_6m='Dosage_6m'
                )
                
                if temporal_results is not None:
                    all_results['temporal_comparisons'][key] = temporal_results
                    all_p_values['temporal'][key] = temporal_results['p_value']
    
    # ===== STAGE 3: CLINICAL PREDICTIVE UTILITY =====
    print("\n\n" + "="*80)
    print("STAGE 3: CLINICAL PREDICTIVE UTILITY")
    print("METHODOLOGY: Mixed models with random intercepts")
    print("="*80)
    
    for hdp_predictor, hdp_type in zip(hdp_predictors_stage3, hdp_types):
        print(f"\n{'='*80}")
        print(f"Analyzing {hdp_type} pathway")
        print(f"{'='*80}")
        
        if hdp_predictor not in df.columns:
            print(f"WARNING: {hdp_predictor} not found in data. Skipping.")
            all_results['clinical_utility'][hdp_predictor] = None
            continue
        
        try:
            utility_results = clinical_predictive_utility_mixed(df, hdp_predictor, hdp_type)
            all_results['clinical_utility'][hdp_predictor] = utility_results
            if utility_results is not None:
                all_p_values['clinical'][hdp_predictor] = utility_results['p_values']
                                    
        except Exception as e:
            print(f"Error in clinical utility analysis for {hdp_type}: {e}")
            all_results['clinical_utility'][hdp_predictor] = None
    
    # ===== APPLY FDR CORRECTION =====
    fdr_results = apply_fdr_correction(all_p_values)
    
    # ===== FINAL SUMMARY WITH BOTH RAW AND FDR P-VALUES =====
    print("\n\n" + "="*80)
    print("FINAL ANALYSIS SUMMARY")
    print("Reporting both raw and FDR-corrected p-values")
    print("="*80)
    
    print("\n1. MEDIATION ANALYSIS RESULTS:")
    print("Format: Effect [95% CI] p-raw (p-FDR)")
    
    for hdp_predictor, hdp_type in zip(hdp_predictors_stage1, hdp_types):
        print(f"\n  {hdp_type} Pathway:")
        for beta_type in ['Total', 'Low', 'High']:
            key = f'{hdp_predictor}_{beta_type}_Beta'
            
            # 3-month results
            if key in all_results['mediation_3m']:
                res = all_results['mediation_3m'][key]
                acme = res['acme']['coeff']
                ci = res['acme']['ci']
                p_raw = res['acme']['pval']
                p_fdr = fdr_results.get('mediation_fdr', {}).get(f"{key}_3m_acme", np.nan)
                sig_raw = '*' if p_raw < 0.05 else ''
                sig_fdr = '†' if p_fdr < 0.05 else ''
                print(f"    {beta_type} Beta (3m) ACME: {acme:.4f} [{ci[0]:.4f}, {ci[1]:.4f}] "
                      f"p={p_raw:.4f}{sig_raw} (FDR={p_fdr:.4f}{sig_fdr})")
            
            # 6-month results
            if key in all_results['mediation_6m']:
                res = all_results['mediation_6m'][key]
                acme = res['acme']['coeff']
                ci = res['acme']['ci']
                p_raw = res['acme']['pval']
                p_fdr = fdr_results.get('mediation_fdr', {}).get(f"{key}_6m_acme", np.nan)
                sig_raw = '*' if p_raw < 0.05 else ''
                sig_fdr = '†' if p_fdr < 0.05 else ''
                print(f"    {beta_type} Beta (6m) ACME: {acme:.4f} [{ci[0]:.4f}, {ci[1]:.4f}] "
                      f"p={p_raw:.4f}{sig_raw} (FDR={p_fdr:.4f}{sig_fdr})")
    
    print("\n2. TEMPORAL COMPARISONS:")
    for key in all_results['temporal_comparisons']:
        if all_results['temporal_comparisons'][key] is not None:
            res = all_results['temporal_comparisons'][key]
            diff = res['difference']
            ci = res['ci']
            p_raw = res['p_value']
            p_fdr = fdr_results.get('temporal_fdr', {}).get(key, np.nan)
            print(f"  {key}: Δ={diff:.4f} [{ci[0]:.4f}, {ci[1]:.4f}] "
                  f"p={p_raw:.4f} (FDR={p_fdr:.4f})")
    
    print("\n3. CLINICAL UTILITY (HDP Predictive Value):")
    for hdp_predictor, hdp_type in zip(hdp_predictors_stage3, hdp_types):
        if all_results['clinical_utility'][hdp_predictor] is not None:
            res = all_results['clinical_utility'][hdp_predictor]
            z_stat = res['z_test']['z_stat']
            p_raw = res['z_test']['p_value']
            p_fdr = fdr_results.get('clinical_fdr', {}).get(f"{hdp_predictor}_mixed", np.nan)
            
            # MSE improvements
            mse_pooled = res['mse_comparison']
            improvement_pooled = (mse_pooled['pooled_model1'] - mse_pooled['pooled_model2']) / mse_pooled['pooled_model1'] * 100
            improvement_3m = (mse_pooled['3_month_model1'] - mse_pooled['3_month_model2']) / mse_pooled['3_month_model1'] * 100 if not np.isnan(mse_pooled['3_month_model1']) else np.nan
            improvement_6m = (mse_pooled['6_month_model1'] - mse_pooled['6_month_model2']) / mse_pooled['6_month_model1'] * 100 if not np.isnan(mse_pooled['6_month_model1']) else np.nan
            
            print(f"  {hdp_type}: Z={z_stat:.3f}, p={p_raw:.4f} (FDR={p_fdr:.4f})")
            print(f"    MSE improvement: Pooled={improvement_pooled:.1f}%, 3m={improvement_3m:.1f}%, 6m={improvement_6m:.1f}%")
    
    print("\n" + "="*80)
    print("ANALYSIS COMPLETE")
    print("* = p < 0.05 (raw), † = p < 0.05 (FDR-corrected)")
    print("="*80)
    
    # Save results
    import pickle
    with open('dbs_analysis_results_corrected.pkl', 'wb') as f:
        pickle.dump({'results': all_results, 'fdr_correction': fdr_results}, f)
    print("\nResults saved to 'dbs_analysis_results_corrected.pkl'")
        
    return all_results, fdr_results

if __name__ == "__main__":
    results, fdr_results = main()